<a href="https://colab.research.google.com/github/galileo15640215/kikagaku/blob/master/kikagaku12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

代表的なモデル
https://www.kikagaku.ai/tutorial/basic_of_computer_vision/learn/tensorflow_advanced_model

In [0]:
import tensorflow as tf
from tensorflow.keras import models, layers

import numpy as np

In [2]:
# Sequential API モデル構築
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(784, )),
    layers.Dense(64, activation='relu')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________


In [0]:
# FunctionalAPI によるモデル構築
inputs = layers.Input(shape=(784, ))

x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(64, activation='relu')(x)

model = models.Model(inputs=inputs, outputs=outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________


In [0]:
def vgg(input_shape, n_classes):
    
    input = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(input)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(2, strides=2, padding='same')(x)
    
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(2, strides=2, padding='same')(x)
    
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(2, strides=2, padding='same')(x)
    
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(2, strides=2, padding='same')(x)
    
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(2, strides=2, padding='same')(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dense(4096, activation='relu')(x)
    
    output = layers.Dense(n_classes, activation='softmax')(x)
    
    model = models.Model(input, output)
    
    return model

In [6]:
model = vgg((224, 224, 3), 1000)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0   

In [0]:
def inception_block(x, f):
    t1 = layers.Conv2D(f[0], 1, activation='relu')(x)
    
    t2 = layers.Conv2D(f[1], 1, activation='relu')(x)
    t2 = layers.Conv2D(f[2], 3, padding='same', activation='relu')(t2)
    
    t3 = layers.Conv2D(f[3], 1, activation='relu')(x)
    t3 = layers.Conv2D(f[4], 5, padding='same', activation='relu')(t3)

    t4 = layers.MaxPool2D(3, 1, padding='same')(x)
    t4 = layers.Conv2D(f[5], 1, activation='relu')(t4)

    out = layers.concatenate([t1, t2, t3, t4])
    
    return out

In [0]:
def googlenet(input_shape, n_classes):
  
    def inception_block(x, f):
        t1 = layers.Conv2D(f[0], 1, activation='relu')(x)

        t2 = layers.Conv2D(f[1], 1, activation='relu')(x)
        t2 = layers.Conv2D(f[2], 3, padding='same', activation='relu')(t2)

        t3 = layers.Conv2D(f[3], 1, activation='relu')(x)
        t3 = layers.Conv2D(f[4], 5, padding='same', activation='relu')(t3)

        t4 = layers.MaxPool2D(3, 1, padding='same')(x)
        t4 = layers.Conv2D(f[5], 1, activation='relu')(t4)

        out = layers.concatenate([t1, t2, t3, t4])

        return out

    input = layers.Input(input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(input)
    x = layers.MaxPool2D(3, strides=2, padding='same')(x)

    x = layers.Conv2D(64, 1, activation='relu')(x)
    x = layers.Conv2D(192, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(3, strides=2)(x)

    x = inception_block(x, [64, 96, 128, 16, 32, 32])
    x = inception_block(x, [128, 128, 192, 32, 96, 64])
    x = layers.MaxPool2D(3, strides=2, padding='same')(x)

    x = inception_block(x, [192, 96, 208, 16, 48, 64])
    x = inception_block(x, [160, 112, 224, 24, 64, 64])
    x = inception_block(x, [128, 128, 256, 24, 64, 64])
    x = inception_block(x, [112, 144, 288, 32, 64, 64])
    x = inception_block(x, [256, 160, 320, 32, 128, 128])
    x = layers.MaxPool2D(3, strides=2, padding='same')(x)

    x = inception_block(x, [256, 160, 320, 32, 128, 128])
    x = inception_block(x, [384, 192, 384, 48, 128, 128])

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)

    output = layers.Dense(n_classes, activation='softmax')(x)

    model = models.Model(input, output)

    return model

In [9]:
model = googlenet((224, 224, 3), 1000)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_13[0][0]                  
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 56, 56, 64)   4160        max_pooling2d_5[0][0]            
____________________________________________________________________________________________

In [0]:
def resnet(input_shape, n_classes):

    def conv_bn_rl(x, f, k=1, s=1, p='same'):
        x = layers.Conv2D(f, k, strides=s, padding=p)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x


    def identity_block(tensor, f):
        x = conv_bn_rl(tensor, f)
        x = conv_bn_rl(x, f, 3)
        x = layers.Conv2D(4*f, 1)(x)
        x = layers.BatchNormalization()(x)

        x = layers.add([x, tensor])
        output = layers.ReLU()(x)
        return output


    def conv_block(tensor, f, s):
        x = conv_bn_rl(tensor, f)
        x = conv_bn_rl(x, f, 3, s)
        x = layers.Conv2D(4*f, 1)(x)
        x = layers.BatchNormalization()(x)

        shortcut = layers.Conv2D(4*f, 1, strides=s)(tensor)
        shortcut = layers.BatchNormalization()(shortcut)

        x = layers.add([x, shortcut])
        output = layers.ReLU()(x)
        return output


    def resnet_block(x, f, r, s=2):
        x = conv_block(x, f, s)
        for _ in range(r-1):
            x = identity_block(x, f)
        return x


    input = layers.Input(input_shape)

    x = conv_bn_rl(input, 64, 7, 2)
    x = layers.MaxPool2D(3, strides=2, padding='same')(x)

    x = resnet_block(x, 64, 3, 1)
    x = resnet_block(x, 128, 4)
    x = resnet_block(x, 256, 6)
    x = resnet_block(x, 512, 3)

    x = layers.GlobalAveragePooling2D()(x)

    output = layers.Dense(n_classes, activation='softmax')(x)

    model = models.Model(input, output)
    return model

In [11]:
model = resnet((224, 224, 3), 1000)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 112, 112, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d_70[0][0]                  
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 112, 112, 64) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [0]:
def mobilenet(input_shape, n_classes):
  
    def mobilenet_block(x, f, strides=1):
        x = layers.DepthwiseConv2D(3, strides=strides, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = layers.Conv2D(f, 1, strides=1, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        return x

    input = layers.Input(input_shape)

    x = layers.Conv2D(32, 3, strides=2, padding='same')(input)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = mobilenet_block(x, 64)
    x = mobilenet_block(x, 128, 2)
    x = mobilenet_block(x, 128)

    x = mobilenet_block(x, 256, 2)
    x = mobilenet_block(x, 256)

    x = mobilenet_block(x, 512, 2)
    for _ in range(5):
        x = mobilenet_block(x, 512)

    x = mobilenet_block(x, 1024, 2)
    x = mobilenet_block(x, 1024)

    x = layers.GlobalAveragePooling2D()(x)

    output = layers.Dense(n_classes, activation='softmax')(x)

    model = models.Model(input, output)

    return model

In [13]:
model = mobilenet((224, 224, 3), 1000)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_53 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_49 (ReLU)              (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_54 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_50 (ReLU)              (None, 112, 112, 32)      0   